In [1]:
%matplotlib inline
from keras.models import Model
from keras.layers import *
from keras.optimizers import *

import keras.initializers as kinit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dqnroute.utils import *

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_csv('logs/data_generated1.csv', names=get_data_cols(10))

In [3]:
data.head()

,time,pkg_id,cur_node,dst_0,dst_1,dst_2,dst_3,dst_4,dst_5,dst_6,...,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,10.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,20.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,20.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-10.0,-1000000.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
3,30.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0
4,30.0,3.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-50.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-1000000.0


In [4]:
target_cols = get_target_cols(10)
neighbors_cols = get_neighbors_cols(10)
addr_cols = get_addr_cols(10)
dst_cols = get_dst_cols(10)

targets = data[target_cols]
neighbors_inp = data[neighbors_cols]
addr_inp = data[addr_cols]
dst_inp = data[dst_cols]

In [5]:
neighbors_inp.head()

,neighbors_0,neighbors_1,neighbors_2,neighbors_3,neighbors_4,neighbors_5,neighbors_6,neighbors_7,neighbors_8,neighbors_9
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [6]:
targets.head()

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,-1000000.0,-1000000.0,-1000000.0,-10.0,-1000000.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
3,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0
4,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-50.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-1000000.0


In [7]:
neighbors_input_dim = neighbors_inp.shape[1]
addr_input_dim = addr_inp.shape[1]
dst_input_dim = dst_inp.shape[1]
output_dim = targets.shape[1]

In [8]:
neighbors_input = Input(shape=(neighbors_input_dim,), name='neighbors_input')
addr_input = Input(shape=(addr_input_dim,), name='addr_input')
dst_input = Input(shape=(dst_input_dim,), name='dst_input')

x = concatenate([neighbors_input, addr_input, dst_input])
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(output_dim, activation='linear')(x)

y = Lambda(lambda x: (1 - x) * -1000000)(neighbors_input)

x = add([x, y])
model = Model(inputs=[neighbors_input, addr_input, dst_input], outputs=x)

model.compile(loss='mse', optimizer='rmsprop')

In [9]:
N_train, N_test, A_train, A_test, D_train, D_test, y_train, y_test = train_test_split(neighbors_inp.values,
                                                                                      addr_inp.values,
                                                                                      dst_inp.values,
                                                                                      targets.values,
                                                                                      test_size=0.2, random_state=42)

In [ ]:
model.fit([N_train, A_train, D_train], y_train, epochs=10)

Epoch 1/10
266440/266440 [==============================] - 11s - loss: 5.1193    
Epoch 2/10
266440/266440 [==============================] - 11s - loss: 0.0527    
Epoch 3/10
266440/266440 [==============================] - 11s - loss: 0.0414    
Epoch 4/10
266440/266440 [==============================] - 11s - loss: 0.0341    
Epoch 5/10
266440/266440 [==============================] - 12s - loss: 0.0291    
Epoch 6/10
266440/266440 [==============================] - 11s - loss: 0.0252    
Epoch 7/10
266440/266440 [==============================] - 11s - loss: 0.0226    
Epoch 8/10
266440/266440 [==============================] - 11s - loss: 0.0205    
Epoch 9/10
117248/266440 [============>.................] - ETA: 6s - loss: 0.0192

In [52]:
model.evaluate([N_test, A_test, D_test], y_test)

65664/66610 [============================>.] - ETA: 0s

0.022835656894448372

In [40]:
model.predict(list(map(lambda x: x.reshape((1, 10)), [N_test[0], A_test[0], D_test[0]])))

array([[ -9.73342037e+00,  -9.99999688e+05,  -9.99999562e+05,
         -1.00000019e+06,  -1.00000000e+06,  -1.00000019e+06,
         -2.98683186e+01,  -9.99999812e+05,  -2.99020615e+01,
         -9.99999812e+05]], dtype=float32)

In [41]:
y_test[0]

array([ -1.00000000e+01,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -3.00000000e+01,  -1.00000000e+06,  -3.00000000e+01,
        -1.00000000e+06])

In [53]:
model.save('router_network.h5')

ImportError: `save_model` requires h5py.